In [1]:
import findspark
findspark.init()
findspark.find()
from operator import add
from pyspark import SparkContext
import wget

Exception: Unable to find py4j in C:\Users\MikeG\OneDrive\Desktop\winutils-master\hadoop-3.0.0\python, your SPARK_HOME may not be configured correctly

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, expr, when,\
desc, first, last, row_number, rank, substring
from pyspark.sql.functions import max as pmax, min as pmin


In [2]:
spark = SparkSession.builder.getOrCreate()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
sc = SparkContext("local[4]", "App1")

In [ ]:
fileName = 'https://people.bu.edu/kalathur/datasets/stocks_2013_2021.txt'
wget.download(fileName)


In [ ]:
stocksFile = sc.textFile("stocks_2013_2021.txt")
stocksFile


In [ ]:
stocksFile.take(2)

In [ ]:
#Problem 1A
mapped = stocksFile.map(lambda x : x.split(','))
mapped.take(2)

In [ ]:
#Problem 1B
mappedVariableTypes = mapped.map(lambda x: [x[0], x[1], float(x[2]), float(x[3]), float(x[4]), float(x[5]), int(x[6])])
mappedVariableTypes.take(2)

In [ ]:
#Problem 1C
mappedVariableTypes.count()

In [ ]:
#Problem 1D
tickerCounts = mappedVariableTypes.map(lambda x: (x[0], 1)).reduceByKey(add).collect()
sorted(tickerCounts, key = lambda x :x[0])

In [ ]:
#Problem 2A
volumeMapped = mappedVariableTypes.map(lambda x: [x[0], x[6]])
volumeMapped.take(2)

In [ ]:
#Problem 2B
volumeTotal = volumeMapped.reduceByKey(add).collect()
volumeTotal

In [ ]:
#Problem 2C
sorted(volumeTotal, key = lambda x : x[1], reverse = True)

In [ ]:
#Problem 2D
sorted(sc.parallelize(volumeTotal).sortBy(lambda x: x[1], ascending = False).collect(), key = lambda x:x[0])

In [ ]:
#Problem 3A
openClose = mappedVariableTypes.map(lambda x: [x[0], (x[2], x[5])])
openClose.take(2)

In [ ]:
#Problem 3B
closeHigherThanOpen = openClose.map(lambda x : (x[0], 1 if x[1][1] > x[1][0] else 0))\
.reduceByKey(add).sortBy(lambda x: x[1], ascending = False).collect()
closeHigherThanOpen

In [ ]:
#Problem 4A
highLow = mappedVariableTypes.map(lambda x: [x[0], (x[3], x[4])])
highLow.take(2)

In [ ]:
#Problem 4B
sorted(highLow.reduceByKey(lambda x, y: (max(x[0], y[0]), min(x[1], y[1]))).collect(), key = lambda x:x[0])


In [ ]:
#Problem 5A
highLowWithDates = mappedVariableTypes.map(lambda x: (x[0], [(x[3], x[1]), (x[4], x[1])]))
highLowWithDates.take(2)


In [ ]:
#Problem 5B
groupedHighLow = highLowWithDates.reduceByKey(lambda x, y : (max(x[0],y[0], key = lambda apples:apples[0]),\
                min(x[1], y[1], key = lambda z : z[0])))
sorted(groupedHighLow.collect())
                                           


In [ ]:
#Problem 6A
stockYearVolume = mappedVariableTypes.map(lambda x: [(x[0], x[1].split('-')[0]), x[6]])
stockYearVolume.take(2)

In [ ]:
stocksByYear = stockYearVolume.reduceByKey(add)
sorted(stocksByYear.collect())

In [ ]:
#Problem 7A
mapHighLow = mappedVariableTypes.map(lambda x: [(x[0], x[1].split('-')[0]), [x[3], x[4]]])
mapHighLow.take(2)

In [ ]:
#Problem 7B
mapRDDHighLowByYear = mapHighLow.reduceByKey(lambda x, y: (max(x[0], y[0]), min(x[1], y[1])))
sorted(mapRDDHighLowByYear.collect(), key = lambda x: x[0])


In [ ]:
#Problem 8A
#Open. High, Low, and Close
mappedVariableTypes.take(1)
googleDates = mappedVariableTypes.filter(lambda x : x[0] == 'GOOG').map(lambda x: (x[1], [x[0], x[2], x[5],\
                                                        round(100*(x[5] - x[2])/x[2], 2)]))
                                                                                          
                                                            
googleDates.take(4)

In [ ]:
#Problem 8B
nflxDates = mappedVariableTypes.filter(lambda x : x[0] == 'NFLX').map(lambda x: (x[1], [x[0], x[2], x[5],\
                                                        round(100*(x[5] - x[2])/x[2], 2)]))
nflxDates.take(4)

In [ ]:
#problem 8C
nflxGoogleJoin =  googleDates.join(nflxDates)
nflxGoogleJoin.takeOrdered(4)

In [ ]:
#Problem 8D
maxGainPerDay = nflxGoogleJoin.map(lambda x: (x[0], max(x[1][0], x[1][1], key = lambda octopus : octopus[3])))
maxGainPerDay.takeOrdered(4)

In [ ]:
#Problem 8E
yearStockGoogle = nflxGoogleJoin.map(lambda x : ((x[0].split('-')[0], x[1][0][0]), x[1][0][3]))
yearStockNetflix = nflxGoogleJoin.map(lambda x : ((x[0].split('-')[0], x[1][1][0]), x[1][1][3]))
yearStockGains = yearStockGoogle.union(yearStockNetflix)
yearStockGains.takeSample(False, 10, seed = 123)

In [ ]:
#Problem 8F 
# I can't find it
collected = yearStockGains.reduceByKey(lambda x, y: (x + y + x*y/100) if x > 0 and y > 0 \
                                      else  (x + y + (x*y/100)) if x > 0 and y < 0 \
                                       else (x + y + x*y/100) if x < 0 and y > 0 \
                                        else (x + y + x*y/100) if x < 0 and y < 0 \
                                         else x)
sorted(collected.collect(), key = lambda strawberries : strawberries[0][0])

                                        